This pipeline currently works tol recreate the 2019 pipeline - yet to be tested with new input.

In the folder is:
- raw_input.csv: the input file with the raw data
- all_ghg_mapping.csv: provides the mapping for the script
- output_for_validation.csv: this was the result in the excel workbook

TO DO:

1. intput_data['2023'] = input_data['2019']
2. Bovenstaande verder verwerken in de code

In [173]:
import pandas as pd

raw_data = pd.read_excel('broeikasgassen_2023_Emissieregistratie.xlsx', usecols=['DATASET','GEMEENTECODE','GEMEENTENAAM','DOELGROEPNAAM','SUBDOELGROEPNAAM','CODE','PROCES_OMSCHRIJVING','STOFCODE','STOFNAAM','EMISSIE_KG'])
print(f"Shape: {raw_data.shape}")
print("\nColumns:", raw_data.columns.tolist())
# print(raw_data['EENHEID'].value_counts())

Shape: (114041, 10)

Columns: ['DATASET', 'CODE', 'PROCES_OMSCHRIJVING', 'SUBDOELGROEPNAAM', 'DOELGROEPNAAM', 'GEMEENTECODE', 'GEMEENTENAAM', 'STOFCODE', 'STOFNAAM', 'EMISSIE_KG']


In [174]:
# DATASET, GEBIEDSINDELING, CODE_GEBIED, GEBIED, DOELGROEP, SUBDOELGROEP, CODE_EMISSIEOORZAAK, EMISSIEOORZAAK, COMPARTIMENT, STOFCODE, STOF, EENHEID, 2O19

input_data = raw_data.rename(columns={'GEMEENTECODE': 'CODE_GEBIED', 'GEMEENTENAAM': 'GEBIED', 'DOELGROEPNAAM':'DOELGROEP', 'SUBDOELGROEPNAAM':'SUBDOELGROEP', 'CODE':'CODE_EMISSIEOORZAAK','PROCES_OMSCHRIJVING':'EMISSIEOORZAAK','STOFNAAM':'STOF','EMISSIE_KG':'2019'})
input_data['EENHEID'] = 'kg'
input_data.head()


,DATASET,CODE_EMISSIEOORZAAK,EMISSIEOORZAAK,SUBDOELGROEP,DOELGROEP,CODE_GEBIED,GEBIED,STOFCODE,STOF,2019,EENHEID
0,ER Reeks 1990-2023 Definitief,0910004,SBI 38: Afvalinzameling en -behandeling (afval...,Overige afvalbedrijven,Afvalverwijdering,1680,Aa en Hunze,205,Distikstofoxide,72.140339,kg
1,ER Reeks 1990-2023 Definitief,0020401,SBI 41-43: Bouwnijverheid,Overig bouw,Bouw,1680,Aa en Hunze,205,Distikstofoxide,1.426587,kg
2,ER Reeks 1990-2023 Definitief,0012102,"Vuurhaarden consumenten, hoofdverwarming woningen",Energiegebruik Consumenten,Consumenten,1680,Aa en Hunze,205,Distikstofoxide,33.544882,kg
3,ER Reeks 1990-2023 Definitief,0800700,"Vuurhaarden consumenten, koken",Energiegebruik Consumenten,Consumenten,1680,Aa en Hunze,205,Distikstofoxide,1.069947,kg
4,ER Reeks 1990-2023 Definitief,0800800,"Vuurhaarden consumenten, warm water voorziening",Energiegebruik Consumenten,Consumenten,1680,Aa en Hunze,205,Distikstofoxide,8.280051,kg


Try to make 'CODE_EMISSIEOORZAAK' with 6 digits such that the mapping improves ---> this was the trick that worked!!\


In [175]:
input_data['CODE_EMISSIEOORZAAK'] = input_data['CODE_EMISSIEOORZAAK'].astype(str).apply(lambda x: x[1:] if x.startswith('0') else x)


In [176]:

def create_co2_scaling_conversion_table():
    """
    Create conversion factors for different units to standardize to metric tons (Mt)

    This table handles the unit conversions needed to convert all emission
    measurements to a common unit (metric tons) before applying GWP factors.

    Returns:
    --------
    pd.DataFrame
        Conversion table with columns: unit_from, unit_to, conversion_factor
    """
    co2_scaling_conversion = pd.DataFrame({
        'unit_from': ['kg', 'ton', 'g', 'mg', 'Gg', 'kt', 'Mt'],
        'unit_to': ['mt', 'mt', 'mt', 'mt', 'mt', 'mt', 'mt'],
        'conversion_factor': [1e-3, 1.0, 1e-6, 1e-9, 1e3, 1e3, 1e6],
        'description': [
            'kilograms to metric tons',
            'tons to metric tons (1:1)',
            'grams to metric tons',
            'milligrams to metric tons',
            'gigagrams to metric tons',
            'kilotons to metric tons',
            'megatons to metric tons'
        ]
    })

    print("CO2 Scaling Conversion Table Created:")
    print(co2_scaling_conversion.to_string(index=False))

    return co2_scaling_conversion

CO2_scaling_conversion_table = create_co2_scaling_conversion_table()

CO2 Scaling Conversion Table Created:
unit_from unit_to  conversion_factor               description
       kg      mt       1.000000e-03  kilograms to metric tons
      ton      mt       1.000000e+00 tons to metric tons (1:1)
        g      mt       1.000000e-06      grams to metric tons
       mg      mt       1.000000e-09 milligrams to metric tons
       Gg      mt       1.000000e+03  gigagrams to metric tons
       kt      mt       1.000000e+03   kilotons to metric tons
       Mt      mt       1.000000e+06   megatons to metric tons


In [177]:
def create_ghg_conversion_table():
    """
    Create Global Warming Potential (GWP) conversion factors for different greenhouse gases

    Based on the specific GWP values provided for this dataset. These factors convert
    the mass of each greenhouse gas to its CO2-equivalent warming potential.

    Returns:
    --------
    pd.DataFrame
        GWP conversion table with greenhouse gas codes, names, and CO2 equivalent factors
    """
    ghg_conversion = pd.DataFrame({
        'greenhouse_gas_code': [205, 904, 687, 906, 688, 903, 696, 204, 523, 641, 909, 908, 409],
        'greenhouse_gas': [
            'Distikstofoxide',                       # N2O
            'HFK-125 (Pentafluorethaan)',           # HFC-125
            'HFK-134a (1,1,1,2-tetrafluorethaan)',  # HFC-134a
            'HFK-143a (1,1,1-Trifluorethaan)',      # HFC-143a
            'HFK-152a (1,1-Difluorethaan)',         # HFC-152a
            'HFK-23 (Trifluormethaan)',             # HFC-23
            'HFK-32 (Difluor-Methaan)',             # HFC-32
            'Koolstofdioxide',                       # CO2
            'Methaan',                               # CH4
            'Overige PFK\'s',                       # Other PFCs
            'PFK 116 (Perfluorethaan)',             # PFC-116
            'PFK 14 (Perfluormethaan)',             # PFC-14
            'Zwavelhexafluoride'                     # SF6
        ],
        'co2_equivalent': [265, 3170, 1300, 4800, 138, 12400, 677, 1, 28, 7500, 11100, 6630, 23500],
        'gas_code_short': ['N2O', 'HFC-125', 'HFC-134a', 'HFC-143a', 'HFC-152a', 'HFC-23',
                          'HFC-32', 'CO2', 'CH4', 'PFC-other', 'PFC-116', 'PFC-14', 'SF6'],
        'gas_category': [
            'N2O', 'F-gas', 'F-gas', 'F-gas', 'F-gas', 'F-gas', 'F-gas',
            'CO2', 'CH4', 'F-gas', 'F-gas', 'F-gas', 'F-gas'
        ],
        'description': [
            'Nitrous oxide',
            'Hydrofluorocarbon-125',
            'Hydrofluorocarbon-134a',
            'Hydrofluorocarbon-143a',
            'Hydrofluorocarbon-152a',
            'Hydrofluorocarbon-23',
            'Hydrofluorocarbon-32',
            'Carbon dioxide (baseline)',
            'Methane',
            'Other perfluorocarbons',
            'Perfluorocarbon C2F6',
            'Perfluorocarbon CF4',
            'Sulfur hexafluoride'
        ]
    })

    print(ghg_conversion[['greenhouse_gas_code', 'greenhouse_gas', 'co2_equivalent']].to_string(index=False))
    return ghg_conversion

GHG_conversion_table = create_ghg_conversion_table()


 greenhouse_gas_code                      greenhouse_gas  co2_equivalent
                 205                     Distikstofoxide             265
                 904          HFK-125 (Pentafluorethaan)            3170
                 687 HFK-134a (1,1,1,2-tetrafluorethaan)            1300
                 906     HFK-143a (1,1,1-Trifluorethaan)            4800
                 688        HFK-152a (1,1-Difluorethaan)             138
                 903            HFK-23 (Trifluormethaan)           12400
                 696            HFK-32 (Difluor-Methaan)             677
                 204                     Koolstofdioxide               1
                 523                             Methaan              28
                 641                       Overige PFK's            7500
                 909            PFK 116 (Perfluorethaan)           11100
                 908            PFK 14 (Perfluormethaan)            6630
                 409                  Zwavelhexaflu

In [178]:
def create_all_ghg_mapping(input_data, ghg_conversion_table, mapping_csv_path='all_ghg_mapping.csv'):
    """
    Create mapping between the gases in raw data and the GWP conversion factors

    This function reads from a CSV file that contains the complete mapping between
    emission sources, gas codes, and gas names, then enriches it with GWP factors.

    Parameters:
    -----------
    input_data : pd.DataFrame
        Raw emissions data
    ghg_conversion_table : pd.DataFrame
        GWP conversion factors table
    mapping_csv_path : str
        Path to the CSV file containing emission-gas mapping

    Returns:
    --------
    pd.DataFrame
        Complete mapping table linking emission sources and gases to GWP factors
    """
    try:
        # Read the mapping CSV file
        mapping_df = pd.read_csv(mapping_csv_path)

        # Clean up column names (remove any empty column)
        mapping_df.columns = [col.strip() if col.strip() else 'empty_col' for col in mapping_df.columns]
        if 'empty_col' in mapping_df.columns:
            mapping_df = mapping_df.drop('empty_col', axis=1)

        # # Also handle 'Unnamed: X' columns
        unnamed_cols = [col for col in mapping_df.columns if col.startswith('Unnamed:')]
        if unnamed_cols:
            mapping_df = mapping_df.drop(unnamed_cols, axis=1)

        # Standardize column names for easier use
        column_mapping = {
            'Emissiecode': 'emission_code',
            'Emissieoorzaak': 'emission_source',
            'Gascode': 'gas_code_raw',
            'Gasnaam': 'gas_name',
            'CRF code': 'crf_code',
            'Emissiecode_Gascode': 'emission_gas_combo',
            'ETM categorie': 'etm_category'
        }

        # Rename columns that exist
        for old_name, new_name in column_mapping.items():
            if old_name in mapping_df.columns:
                mapping_df = mapping_df.rename(columns={old_name: new_name})

        # Merge with GWP conversion table to add CO2 equivalent factors
        all_ghg_mapping = mapping_df.merge(
            ghg_conversion_table[['greenhouse_gas', 'co2_equivalent', 'gas_code_short', 'gas_category']],
            left_on='gas_name',
            right_on='greenhouse_gas',
            how='left'
        )

        # Handle unmapped gases
        unmapped_mask = all_ghg_mapping['co2_equivalent'].isnull()
        unmapped_count = unmapped_mask.sum()

        if unmapped_count > 0:
            all_ghg_mapping.loc[unmapped_mask, 'co2_equivalent'] = 1
            all_ghg_mapping.loc[unmapped_mask, 'gas_code_short'] = 'OTHER'
            all_ghg_mapping.loc[unmapped_mask, 'gas_category'] = 'OTHER'

        print(f"\nSample mapping (first 10 rows):")
        print(all_ghg_mapping.head(10).to_string(index=False))

        return all_ghg_mapping

    except FileNotFoundError:
        print(f"  Error: Could not find mapping file '{mapping_csv_path}'")
        print("   Falling back to simple gas-based mapping...")

        # Fallback to original method if CSV not found
        unique_gases = input_data['STOF'].unique()

        all_ghg_mapping = pd.DataFrame({
            'gas_name': unique_gases
        })

        # Map to conversion table
        all_ghg_mapping = all_ghg_mapping.merge(
            ghg_conversion_table[['greenhouse_gas', 'co2_equivalent', 'gas_code_short', 'gas_category']],
            left_on='gas_name',
            right_on='greenhouse_gas',
            how='left'
        )

        # Handle unmapped gases
        unmapped_mask = all_ghg_mapping['co2_equivalent'].isnull()
        all_ghg_mapping.loc[unmapped_mask, 'co2_equivalent'] = 1
        all_ghg_mapping.loc[unmapped_mask, 'gas_code_short'] = 'OTHER'
        all_ghg_mapping.loc[unmapped_mask, 'gas_category'] = 'OTHER'

        return all_ghg_mapping

    except Exception as e:
        print(f"  Error reading mapping file: {str(e)}")
        raise

# EXECUTE THIS CELL:
all_GHG_mapping = create_all_ghg_mapping(input_data, GHG_conversion_table)


Sample mapping (first 10 rows):
emission_code                                                                                            emission_source  gas_code_raw        gas_name         crf_code emission_gas_combo                                         etm_category  greenhouse_gas  co2_equivalent gas_code_short gas_category
        12100                                                                     SBI: Huishoudens, verbrandingsemissies           204 Koolstofdioxide        1.A.4.b.i          12100_204 NA (energetische emissies, volgt uit energiegebruik) Koolstofdioxide             1.0            CO2          CO2
        12100                                                                     SBI: Huishoudens, verbrandingsemissies           205 Distikstofoxide        1.A.4.b.i          12100_205             energetic_emissions_other_ghg_households Distikstofoxide           265.0            N2O          N2O
        12100                                                      

In [179]:
def create_all_ghg_conversion(input_data, all_ghg_mapping, co2_scaling_conversion_table):
    """
    Apply all conversions to the raw data to get CO2 equivalents in metric tons

    This is the core conversion step that:
    1. Adds GWP factors to each emission record
    2. Adds unit conversion factors
    3. Calculates CO2 equivalent emissions in metric tons

    Formula: CO2_eq_mt = Original_emission × GWP_factor × Unit_conversion_factor

    Parameters:
    -----------
    input_data : pd.DataFrame
        Raw emissions data
    all_ghg_mapping : pd.DataFrame
        Mapping between substances and GWP factors
    co2_scaling_conversion_table : pd.DataFrame
        Unit conversion factors

    Returns:
    --------
    pd.DataFrame
        Complete dataset with CO2 equivalent calculations
    """
    print("Applying conversions to raw data...")

    # Start with raw data
    all_ghg_conversion = input_data.copy()

    # Create lookup key for mapping (emission code + gas code)
    all_ghg_conversion['lookup_key'] = all_ghg_conversion['CODE_EMISSIEOORZAAK'].astype(str) + '_' + all_ghg_conversion['STOFCODE'].astype(str)

    # Add GWP factors using the emission-gas combination mapping
    all_ghg_conversion = all_ghg_conversion.merge(
        all_ghg_mapping[['emission_gas_combo', 'co2_equivalent', 'gas_code_short', 'gas_category', 'etm_category']],
        left_on='lookup_key',
        right_on='emission_gas_combo',
        how='left'
    )

    # For records not found in detailed mapping, fall back to simple gas mapping
    missing_mapping = all_ghg_conversion['co2_equivalent'].isnull()
    if missing_mapping.any():
        print(f"   {missing_mapping.sum()} records not found in detailed mapping, using simple gas mapping...")

        # Create simple gas mapping fallback
        simple_mapping = all_ghg_mapping[['gas_name', 'co2_equivalent', 'gas_code_short', 'gas_category']].drop_duplicates('gas_name')

        # Fill missing values using simple gas name mapping
        missing_data = all_ghg_conversion[missing_mapping].copy()
        missing_data = missing_data.merge(
            simple_mapping,
            left_on='STOF',
            right_on='gas_name',
            how='left',
            suffixes=('', '_fallback')
        )

        # Update the missing records
        all_ghg_conversion.loc[missing_mapping, 'co2_equivalent'] = missing_data['co2_equivalent_fallback'].fillna(1)
        all_ghg_conversion.loc[missing_mapping, 'gas_code_short'] = missing_data['gas_code_short_fallback'].fillna('OTHER')
        all_ghg_conversion.loc[missing_mapping, 'gas_category'] = missing_data['gas_category_fallback'].fillna('OTHER')
        all_ghg_conversion.loc[missing_mapping, 'etm_category'] = 'Unknown'

    # Add unit conversion factors
    all_ghg_conversion = all_ghg_conversion.merge(
        co2_scaling_conversion_table[['unit_from', 'conversion_factor']],
        left_on='EENHEID',
        right_on='unit_from',
        how='left'
    )

    # Fill missing conversion factors (assume kg if not found)
    missing_units = all_ghg_conversion['conversion_factor'].isnull()
    if missing_units.any():
        unique_missing = all_ghg_conversion[missing_units]['EENHEID'].unique()
        print(f"   Missing conversion factors for units: {unique_missing} (assuming kg)")
        all_ghg_conversion.loc[missing_units, 'conversion_factor'] = 1e-3

    # Calculate CO2 equivalent in metric tons for 2019
    all_ghg_conversion['CO2 equivalent MT'] = (
        all_ghg_conversion['2019'].fillna(0) *          # Original emission value
        all_ghg_conversion['co2_equivalent'] *          # Global Warming Potential
        all_ghg_conversion['conversion_factor'] /       # Unit conversion to metric tons
        1000000                                         # Final scaling factor for MT CO₂-eq
    )

    # Add the combination code column
    all_ghg_conversion['CODE_EMISSIEOORZAAK_STOFCODE'] = all_ghg_conversion['CODE_EMISSIEOORZAAK'].astype(str) + '_' + all_ghg_conversion['STOFCODE'].astype(str)

    # Rename ETM category column to match expected format
    all_ghg_conversion = all_ghg_conversion.rename(columns={'etm_category': 'ETM category'})

    print(f"   Conversion completed successfully!")
    print(f"   Dataset shape: {all_ghg_conversion.shape}")

    # Quality checks
    total_co2_eq = all_ghg_conversion['CO2 equivalent MT'].sum()
    print(f"   Total CO2 equivalent emissions (2019): {total_co2_eq:,.6f} Mt CO2-eq")

    # Show sample with expected format
    print("\nSample converted values (expected format):")
    expected_cols = [
        'DATASET', 'GEBIEDSINDELING', 'CODE_GEBIED', 'GEBIED', 'DOELGROEP', 'SUBDOELGROEP',
        'CODE_EMISSIEOORZAAK', 'EMISSIEOORZAAK', 'COMPARTIMENT', 'STOFCODE', 'STOF',
        'EENHEID', '2019', 'CO2 equivalent MT', 'CODE_EMISSIEOORZAAK_STOFCODE', 'ETM category'
    ]

    # Show only columns that exist
    available_cols = [col for col in expected_cols if col in all_ghg_conversion.columns]
    sample_data = all_ghg_conversion[available_cols].head(5)
    print(sample_data.to_string(index=False))

    # Show breakdown by gas category
    print(f"\nBreakdown by gas category:")
    if 'gas_category' in all_ghg_conversion.columns:
        category_summary = all_ghg_conversion.groupby('gas_category')['CO2 equivalent MT'].sum().sort_values(ascending=False)
        for category, total in category_summary.items():
            percentage = (total / total_co2_eq) * 100 if total_co2_eq > 0 else 0
            print(f"   {category}: {total:,.6f} Mt CO2-eq ({percentage:.1f}%)")

    return all_ghg_conversion

all_GHG_conversion = create_all_ghg_conversion(input_data, all_GHG_mapping, CO2_scaling_conversion_table)

Applying conversions to raw data...
   29200 records not found in detailed mapping, using simple gas mapping...
   Conversion completed successfully!
   Dataset shape: (114592, 21)
   Total CO2 equivalent emissions (2019): 667.195516 Mt CO2-eq

Sample converted values (expected format):
                       DATASET  CODE_GEBIED      GEBIED         DOELGROEP               SUBDOELGROEP CODE_EMISSIEOORZAAK                                                  EMISSIEOORZAAK  STOFCODE            STOF EENHEID      2019  CO2 equivalent MT CODE_EMISSIEOORZAAK_STOFCODE                             ETM category
ER Reeks 1990-2023 Definitief          1680 Aa en Hunze Afvalverwijdering     Overige afvalbedrijven              910004 SBI 38: Afvalinzameling en -behandeling (afvalwaterbehandeling)       205 Distikstofoxide      kg 72.140339       1.911719e-05                   910004_205                                  Unknown
ER Reeks 1990-2023 Definitief          1680 Aa en Hunze              Bouw   

In [180]:
def create_other_ghg_co2_eq_mt(all_ghg_conversion):
    """
    Create final output dataframe with non-CO2 GHG emissions in CO2 equivalents

    This function:
    1. Filters out CO2 emissions to focus on "other" greenhouse gases
    2. Aggregates emissions by municipality and ETM category (not administrative dimensions)
    3. Pivots to wide format matching reference data structure

    Parameters:
    -----------
    all_ghg_conversion : pd.DataFrame
        Complete dataset with CO2 equivalent calculations

    Returns:
    --------
    pd.DataFrame
        Final aggregated other GHG emissions in Mt CO2-eq by municipality and ETM category
    """
    print("Creating final output: other_GHG_co2_eq_mt")

    # Filter out CO2 to get "other" GHGs only
    # other_ghg_conversion = all_ghg_conversion[all_ghg_conversion['STOF'] != 'Koolstofdioxide'].copy() ###CHANGED
    # Exclude only 'energetic_emissions_co2' – keep relevant CO2 categories like indirect and non-energetic
    excluded_categories = ['energetic_emissions_co2']
    other_ghg_conversion = all_ghg_conversion[~all_ghg_conversion['ETM category'].isin(excluded_categories)].copy()

    print(f"Filtered dataset: {len(other_ghg_conversion)} records (excluding CO2)")

    # Check what ETM categories we have
    etm_categories = other_ghg_conversion['ETM category'].value_counts()
    print(f"\nETM categories found:")
    for category, count in etm_categories.items():
        if pd.notna(category):
            print(f"   {category}: {count} records")

    # Remove records with missing ETM category
    other_ghg_conversion = other_ghg_conversion[other_ghg_conversion['ETM category'].notna()]
    print(f"After removing missing ETM categories: {len(other_ghg_conversion)} records")

    # Aggregate by municipality and ETM category (this is the key fix!)
    etm_aggregation = other_ghg_conversion.groupby(['CODE_GEBIED','GEBIED', 'ETM category'])['CO2 equivalent MT'].sum().reset_index()

    # Pivot to wide format to match reference data structure
    other_ghg_co2_eq_mt = etm_aggregation.pivot(
        index=['CODE_GEBIED','GEBIED'],
        columns='ETM category',
        values='CO2 equivalent MT'
    # ).fillna(0).reset_index()
    ).fillna(0)
    # Reset index zodat 'CODE_GEBIED' en 'GEBIED' gewone kolommen worden
    other_ghg_co2_eq_mt = other_ghg_co2_eq_mt.reset_index()

    # Calculate CO2 equivalent in metric tons for 2019
    all_ghg_conversion['CO2 equivalent MT'] = (
        all_ghg_conversion['2019'].fillna(0) *          # Original emission value
        all_ghg_conversion['co2_equivalent'] *          # Global Warming Potential
        all_ghg_conversion['conversion_factor'] /       # Unit conversion to metric tons
        1000000                                         # Final scaling factor for MT CO₂-eq
    )

    # Add logic for 'non_energetic_emissions_other_ghg_agriculture_manure'
    manure_filter = all_ghg_conversion['ETM category'] == 'non_energetic_emissions_other_ghg_agriculture_manure'
    if manure_filter.any():
        manure_emissions = all_ghg_conversion.loc[manure_filter, 'CO2 equivalent MT'].sum()
        print(f"   Total emissions for 'non_energetic_emissions_other_ghg_agriculture_manure': {manure_emissions:,.6f} Mt CO2-eq")
    else:
        print("   No emissions found for 'non_energetic_emissions_other_ghg_agriculture_manure'.")

    # Add the combination code column
    all_ghg_conversion['CODE_EMISSIEOORZAAK_STOFCODE'] = all_ghg_conversion['CODE_EMISSIEOORZAAK'].astype(str) + '_' + all_ghg_conversion['STOFCODE'].astype(str)

    # Rename the municipality column to match reference format
    other_ghg_co2_eq_mt = other_ghg_co2_eq_mt.rename(columns={'CODE_GEBIED': 'GemeenteCode', 'GEBIED': 'Gemeentenaam'})

    other_ghg_co2_eq_mt['GemeenteCode'] = other_ghg_co2_eq_mt['GemeenteCode'].astype(str).str.zfill(4)
    other_ghg_co2_eq_mt['GemeenteCode'] = 'GM' + other_ghg_co2_eq_mt['GemeenteCode']


    # Add any missing columns that exist in reference but not in our data
    reference_columns = [
        'energetic_emissions_other_ghg_agriculture',
        'energetic_emissions_other_ghg_buildings',
        'energetic_emissions_other_ghg_energy',
        'energetic_emissions_other_ghg_households',
        'energetic_emissions_other_ghg_industry',
        'energetic_emissions_other_ghg_transport',
        'non_energetic_emissions_other_ghg_agriculture_fermentation',
        'non_energetic_emissions_other_ghg_agriculture_manure',
        'non_energetic_emissions_other_ghg_agriculture_other',
        'non_energetic_emissions_other_ghg_agriculture_soil_cultivation',
        'non_energetic_emissions_other_ghg_chemical_industry',
        'non_energetic_emissions_other_ghg_other_industry',
        'non_energetic_emissions_other_ghg_waste_management',
        'indirect_emissions_co2',
        'non_energetic_emissions_co2_agriculture_manure',
        'non_energetic_emissions_co2_agriculture_soil_cultivation',
        'non_energetic_emissions_co2_chemical_industry',
        'non_energetic_emissions_co2_other_industry',
        'non_energetic_emissions_co2_waste_management'
    ]

    # Add missing columns with zero values
    for col in reference_columns:
        if col not in other_ghg_co2_eq_mt.columns:
            other_ghg_co2_eq_mt[col] = 0.0

    # Reorder columns to match reference format (Municipality first, then ETM categories)
    existing_etm_columns = [col for col in reference_columns if col in other_ghg_co2_eq_mt.columns]
    column_order = ['GemeenteCode', 'Gemeentenaam'] + existing_etm_columns
    other_ghg_co2_eq_mt = other_ghg_co2_eq_mt[column_order]

    # Summary statistics
    total_other_ghg = other_ghg_co2_eq_mt[existing_etm_columns].sum().sum()

    print(f"\n DISTRIBUTION:")
    category_totals = other_ghg_co2_eq_mt[existing_etm_columns].sum().sort_values(ascending=False)
    for category, total in category_totals.items():
        if total > 0:
            percentage = (total / total_other_ghg) * 100
            print(f"   {category}: {total:,.6f} Mt CO2-eq ({percentage:.1f}%)")

    print(f"\n TOP 10 MUNICIPALITIES:")
    other_ghg_co2_eq_mt['total'] = other_ghg_co2_eq_mt[existing_etm_columns].sum(axis=1)
    top_municipalities = other_ghg_co2_eq_mt.nlargest(10, 'total')
    for i, (_, row) in enumerate(top_municipalities.iterrows(), 1):
        print(f"   {i:2d}. {row['GemeenteCode']}: {row['total']:,.6f} Mt CO2-eq")

    # Remove the temporary total column
    other_ghg_co2_eq_mt = other_ghg_co2_eq_mt.drop('total', axis=1)

    # Show sample of final output
    print(f"\nSample of final output (first 5 municipalities):")
    sample_cols = ['GemeenteCode'] + existing_etm_columns[:5]
    available_sample_cols = [col for col in sample_cols if col in other_ghg_co2_eq_mt.columns]
    print(other_ghg_co2_eq_mt[available_sample_cols].head().to_string(index=False))

    return other_ghg_co2_eq_mt

other_GHG_co2_eq_mt = create_other_ghg_co2_eq_mt(all_GHG_conversion)

Creating final output: other_GHG_co2_eq_mt
Filtered dataset: 114592 records (excluding CO2)

ETM categories found:
   Unknown: 29200 records
   NA (energetische emissies, volgt uit energiegebruik): 20118 records
   energetic_emissions_other_ghg_transport: 15998 records
   energetic_emissions_other_ghg_industry: 10227 records
   out_of_scope: 5967 records
   non_energetic_emissions_other_ghg_other_industry: 4826 records
   non_energetic_emissions_other_ghg_agriculture_manure: 4766 records
   non_energetic_emissions_other_ghg_agriculture_soil_cultivation: 3779 records
   energetic_emissions_other_ghg_households: 3436 records
   indirect_emissions_co2: 3096 records
   non_energetic_emissions_other_ghg_waste_management: 2719 records
   non_energetic_emissions_co2_other_industry: 2427 records
   non_energetic_emissions_other_ghg_agriculture_fermentation: 2070 records
   energetic_emissions_other_ghg_agriculture: 1938 records
   energetic_emissions_other_ghg_buildings: 1225 records
   energe

Notes: 

The script drops 17 categories which were not included in the 2019 pipeline - those are the 930042_523 (Aviation LTO Idle + Methane) categories which we could map to energetic_emissions_other_ghg_transport. Should these be included? 

In addition, the original pipeline didn't include Nordzee. This pipeline does - should it exclude Nordzee?

In [181]:
other_GHG_co2_eq_mt.head()

# Save the final output to a CSV file
output_file = other_GHG_co2_eq_mt.to_csv('non_energetic_emissions_output.csv', index=False)



In [182]:

# Bestandsnamen
bestand1 = 'output_2019.csv'
bestand2 = 'non_energetic_emissions_output.csv'
output = 'comparison_2023_2019.csv'

# Lees CSV-bestanden in met eerste kolom als index (bijv. 'Gemeente')
df1 = pd.read_csv(bestand1, index_col=0)
df2 = pd.read_csv(bestand2, index_col=0)

# Unie van alle rijen en kolommen
alle_rijen = df1.index.union(df2.index)
alle_kolommen = df1.columns.union(df2.columns)

# Reindex om verschillen op te vangen
df1 = df1.reindex(index=alle_rijen, columns=alle_kolommen)
df2 = df2.reindex(index=alle_rijen, columns=alle_kolommen)

# Resultaat dataframe
resultaat = pd.DataFrame(index=alle_rijen, columns=alle_kolommen)

# Vergelijk en bereken val1 / val2
for row in alle_rijen:
    for col in alle_kolommen:
        val1 = df1.at[row, col]
        val2 = df2.at[row, col]

        try:
            if pd.isna(val1) and pd.isna(val2):
                resultaat.at[row, col] = ""
            elif pd.isna(val1):
                resultaat.at[row, col] = f"alleen in bestand2"
            elif pd.isna(val2):
                resultaat.at[row, col] = f"alleen in bestand1"
            elif float(val2) == 0:
                resultaat.at[row, col] = "delen door nul"
            else:
                ratio = float(val1) / float(val2)
                resultaat.at[row, col] = round(ratio, 6)  # Afronden op 6 decimalen
        except Exception as e:
            resultaat.at[row, col] = f"fout: {e}"

# Opslaan
resultaat.to_csv(output)
print(f"✅ Ratio-bestand opgeslagen als '{output}'")

✅ Ratio-bestand opgeslagen als 'comparison_2023_2019.csv'


In [183]:
# COMMIT MESSAGES VOOR ALLE KOLOMMEN IN OUTPUT.CSV
print("=== COMMIT MESSAGES VOOR ALLE NON-ENERGETIC EMISSIONS KOLOMMEN ===")

# Lees de output om de kolommen te zien
import pandas as pd
output_df = pd.read_csv('non_energetic_emissions_output.csv')
columns = [col for col in output_df.columns if col != 'Municipality']

print(f"Kolommen in output.csv: {len(columns)}")
for i, col in enumerate(columns, 1):
    print(f"{i:2d}. {col}")

print(f"\n=== COMMIT MESSAGES ===")

# Commit messages per kolom
commit_messages = {
    "energetic_emissions_other_ghg_agriculture": 
        "Energetic other greenhouse gas emissions from agriculture are calculated by converting agricultural combustion N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "energetic_emissions_other_ghg_buildings": 
        "Energetic other greenhouse gas emissions from buildings are calculated by converting commercial and public building combustion N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "energetic_emissions_other_ghg_energy": 
        "Energetic other greenhouse gas emissions from energy sector are calculated by converting oil/gas extraction and power generation N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "energetic_emissions_other_ghg_households": 
        "Energetic other greenhouse gas emissions from households are calculated by converting residential combustion N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "energetic_emissions_other_ghg_industry": 
        "Energetic other greenhouse gas emissions from industry are calculated by converting industrial combustion N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "energetic_emissions_other_ghg_transport": 
        "Energetic other greenhouse gas emissions from transport are calculated by converting vehicle exhaust N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_agriculture_fermentation": 
        "Non-energetic agriculture fermentation emissions are calculated by converting livestock enteric fermentation CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_agriculture_manure": 
        "Non-energetic agriculture manure emissions are calculated by converting livestock manure N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_agriculture_other": 
        "Non-energetic other agriculture emissions are calculated by converting miscellaneous agricultural process N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_agriculture_soil_cultivation": 
        "Non-energetic agriculture soil cultivation emissions are calculated by converting agricultural soil N2O emissions from fertilizer application (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_chemical_industry": 
        "Non-energetic chemical industry emissions are calculated by converting industrial process F-gas, N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_other_industry": 
        "Non-energetic other industry emissions are calculated by converting miscellaneous industrial process N2O, CH4 and F-gas emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors.",
    
    "non_energetic_emissions_other_ghg_waste_management": 
        "Non-energetic waste management emissions are calculated by converting waste treatment and landfill N2O and CH4 emissions (Emissieregistratie, 2023) to CO2-equivalents using IPCC GWP factors."
}

# Print alle commit messages
for i, (column, message) in enumerate(commit_messages.items(), 1):
    print(f"\n{i:2d}. {column}:")
    print(f'    "{message}"')

# Verificatie - check of alle kolommen gedekt zijn
missing_columns = set(columns) - set(commit_messages.keys())
if missing_columns:
    print(f"\n⚠️  MISSENDE KOLOMMEN: {missing_columns}")
else:
    print(f"\n✅ ALLE {len(columns)} KOLOMMEN HEBBEN COMMIT MESSAGES")

# Export naar CSV voor gemakkelijk gebruik
commit_df = pd.DataFrame([
    {"column": col, "commit_message": msg} 
    for col, msg in commit_messages.items()
])
commit_df.to_csv('commit_messages_non_energetic_emissions.csv', index=False)
print(f"\n📁 Commit messages geëxporteerd naar: commit_messages_non_energetic_emissions.csv")

=== COMMIT MESSAGES VOOR ALLE NON-ENERGETIC EMISSIONS KOLOMMEN ===
Kolommen in output.csv: 21
 1. GemeenteCode
 2. Gemeentenaam
 3. energetic_emissions_other_ghg_agriculture
 4. energetic_emissions_other_ghg_buildings
 5. energetic_emissions_other_ghg_energy
 6. energetic_emissions_other_ghg_households
 7. energetic_emissions_other_ghg_industry
 8. energetic_emissions_other_ghg_transport
 9. non_energetic_emissions_other_ghg_agriculture_fermentation
10. non_energetic_emissions_other_ghg_agriculture_manure
11. non_energetic_emissions_other_ghg_agriculture_other
12. non_energetic_emissions_other_ghg_agriculture_soil_cultivation
13. non_energetic_emissions_other_ghg_chemical_industry
14. non_energetic_emissions_other_ghg_other_industry
15. non_energetic_emissions_other_ghg_waste_management
16. indirect_emissions_co2
17. non_energetic_emissions_co2_agriculture_manure
18. non_energetic_emissions_co2_agriculture_soil_cultivation
19. non_energetic_emissions_co2_chemical_industry
20. non_energ